# Xorshift Random Number Generator in Sagemath/Jupyter, @iwaokimura

G. Marsagliaの乱数生成器xorshiftを，sagemathで試てみる．この乱数は，2元体上の $N$ 次元ベクトル空間 $V$ のある元 $x$ を初期値として選び，$V$ 上の線形変換 $T$ を $x$ に繰り返し施した値 $T^m(x)$ を生成した乱数値とするものである．

2元体での演算が xor と同じであることにまず注意しよう．ビット列についても同様に，ビット列の xor と，2元体上のベクトル空間の成分の和が同じものになる．更に，ビット列の左，右シフトが，2元体上のベクトル空間の線形変換 $L, R$（後述）になる．以上から，線形変換 $T$ として，次のような特別なものに注目することを思いつく：
$T$ を，$a, b, c$ を適切に選んで， $T=(1+L^a)(1+R^b)(1+L^c)$ とする．「適切に」というのは，$T$ の（$V$ 上の線形変換としての）位数が十分大きくなるように（正確には，$T$ の位数が $2^N-1$ になるように），という意味である．

以上の詳細については，「Google Chromeが採用した、擬似乱数生成アルゴリズム「xorshift」の数理」https://blog.visvirial.com/articles/575 が大変読みやすい解説である．同記事では，Python3でのコードが掲載されている．本稿の目的は，同じことをsagemathで行うと，ほぼ何も実装せずに済む，ということのデモである．Sagemathは今すぐに https://cloud.sagemath.com/ で始めることができる．

以下では，$N=32$ として，$T$ の位数が $2^N-1$ となる $a, b, c$ の探索を行う．これは，G. Marsaglia, Xorshift RNGs, http://dx.doi.org/10.18637/jss.v008.i14 のはじめの2ページほどに相当する．

In [9]:
N=32; # 次元

In [2]:
G = GL(N, FiniteField(2)); G # 2元体上の，N 次正則（可逆）行列の群

General Linear Group of degree 32 over Finite Field of size 2

In [3]:
G.order() # 試しに G の位数を計算する．

51915237608802545834650365775747737098229791081502830361375666998864635954655274189188618336673413249941113660054889860892524028523586055095785930107241175599128831839106952836064314134436759863399605193683057244940843342962051291562930060275608907956668714644423971391788961835934547494654875533312000000000

In [4]:
G.order().factor() # 素因数分解してみる

2^496 * 3^22 * 5^9 * 7^11 * 11^3 * 13^2 * 17^4 * 19 * 23^2 * 29 * 31^6 * 41 * 43^2 * 47 * 73^3 * 89^2 * 113 * 127^4 * 151^2 * 233 * 241 * 257^2 * 331 * 337 * 601 * 683 * 1103 * 1801 * 2089 * 2731 * 8191^2 * 65537 * 131071 * 178481 * 262657 * 524287 * 2147483647

$G$ はこのあとは登場しないが，以下で現れる $I+L^a$, $I+R^b$ などや，それらの積である $T$ はいずれも正則行列（可逆行列）で，$G$ の元である．

まず，左シフトに相当する行列 $L$ の成分を与える関数を用意する．(1,2), (2, 3), ..., (31, 32)成分が1, あとはすべて0である行列である．つまり，i行j列の成分が，i+1 == j のとき1, それ以外の時 0 である．ただし，sagemathでは行列，ベクトルは0番目の成分から数えることに注意する．

In [1]:
def Lcoeff(i, j):
    if i+1 == j:
        return FiniteField(2).one()
    else:
        return FiniteField(2).zero()

試しに数字を列挙してみる（出力は長いので省略）：

In [ ]:
for i in range(0, 32):
    for j in range(0, 32):
        print(i, j, Lcoeff(i,j))

上の関数 Lcoeff() を元に，左シフトに対応する行列Lを作る：

In [6]:
L = matrix([[Lcoeff(i,j) for j in range(32)] for i in range(32)])

In [7]:
type(L)

<type 'sage.matrix.matrix_mod2_dense.Matrix_mod2_dense'>

遅ればせながらここで単位行列を用意する：

In [14]:
idelem = matrix.identity(FiniteField(2), N) # id element

以下，右シフトについて同様のことをする．

In [11]:
def Rcoeff(i, j):
    if i == j+1:
        return FiniteField(2).one()
    else:
        return FiniteField(2).zero()

In [12]:
R = matrix([[Rcoeff(i,j) for j in range(32)] for i in range(32)])

あとは，$T=(1+L^s)(1+R^t)(1+L^u)$ の位数を，$s \le u$ の範囲でループを回しながら計算し，$2^N-1$ になるような $s, t, u$ を列挙するだけ．

In [22]:
@parallel(p_iter='fork', ncpus=4)
def make_list():
    c = 0
    for s in range(1,32):
        for t in range(1, 32):
            for u in range(s, 32):
                if ((idelem+L^s)*(idelem+R^t)*(idelem+L^u)).multiplicative_order() == 2^N-1:
                    c += 1
                    print(s, t, u)
    print("total number = ", c)

In [23]:
make_list()

(1, 3, 10)
(1, 5, 16)
(1, 5, 19)
(1, 9, 29)
(1, 11, 6)
(1, 11, 16)
(1, 19, 3)
(1, 21, 20)
(1, 27, 27)
(2, 5, 15)
(2, 5, 21)
(2, 7, 7)
(2, 7, 9)
(2, 7, 25)
(2, 9, 15)
(2, 15, 17)
(2, 15, 25)
(2, 21, 9)
(3, 1, 14)
(3, 3, 26)
(3, 3, 28)
(3, 3, 29)
(3, 5, 20)
(3, 5, 22)
(3, 5, 25)
(3, 7, 29)
(3, 13, 7)
(3, 23, 25)
(3, 25, 24)
(3, 27, 11)
(4, 3, 17)
(4, 3, 27)
(4, 5, 15)
(5, 3, 21)
(5, 7, 22)
(5, 9, 7)
(5, 9, 28)
(5, 9, 31)
(5, 13, 6)
(5, 15, 17)
(5, 17, 13)
(5, 21, 12)
(5, 27, 8)
(5, 27, 21)
(5, 27, 25)
(5, 27, 28)
(6, 1, 11)
(6, 3, 17)
(6, 17, 9)
(6, 21, 7)
(6, 21, 13)
(7, 1, 9)
(7, 1, 18)
(7, 1, 25)
(7, 13, 25)
(7, 17, 21)
(7, 25, 12)
(7, 25, 20)
(8, 7, 23)
(8, 9, 23)
(9, 5, 14)
(9, 5, 25)
(9, 11, 19)
(9, 21, 16)
(10, 9, 21)
(10, 9, 25)
(11, 7, 12)
(11, 7, 16)
(11, 17, 13)
(11, 21, 13)
(12, 9, 23)
(13, 3, 17)
(13, 3, 27)
(13, 5, 19)
(13, 17, 15)
(14, 1, 15)
(14, 13, 15)
(15, 1, 29)
(17, 15, 20)
(17, 15, 23)
(17, 15, 26)
('total number = ', 81)


以上．